    1. Enter Hash tag
    2. Extract 300 user per hashtag
    3. Check whether those users are in UsersData table. 
    4. Extract for only those whose data is not in the table.
    5. Recall back the userdata into dataframe, where date is today's date. 



In [1]:
# Step 1
# Enter today's hashtag !, don't put # , just put the text. 
hashtags = ['cute', 'inspiration', 'tbt', 'like4like', 'followme', 'kualalumpur', 'hongkong', 'vietnam', 'london', 'milan']
user_pwd = {'rat163001': 'Abbi@163', 'rat163002' : 'Abbi@163', 'rat163004': 'Abbi@163'}


In [2]:


# Change directory
import os
os.chdir('C:\\Users\\PC\\Desktop\\Instagram_production')

# Necessary imports
from datetime import datetime, timedelta
from itertools import dropwhile, takewhile
from instaloader import Instaloader, Profile
from src.insta import InstagramBot
from sklearn.externals.joblib import load
from src.FeatureExtraction import featureExtraction
import sqlite3 as db
import numpy as np
import pandas as pd
from numpy import random
from selenium.common.exceptions import TimeoutException

for key in user_pwd.keys():
    try:
        t = random.randint(0, len(hashtags))
        hashtag = hashtags[t]

        # Step 2
        L = Instaloader()
        data = []
        posts = L.get_hashtag_posts(hashtag)
        likes = set()

        today = datetime.today()
        tomorrow = datetime.today() + timedelta(days = 2)
        yesterday = datetime.today() + timedelta(days = -1)

        # Since is big | less past 
        SINCE = tomorrow
        # Until is small | past
        UNTIL = yesterday

        # Extract the list of people to send request to, note : We are selecting the users who are recently active. 

        for post in takewhile(lambda p: p.date > UNTIL, dropwhile(lambda p: p.date > SINCE, posts)):
            print(post.date)
            likes = post.get_likes()
            for like in likes:
                data.append(like.username)
            users = np.unique(data)
            if len(users) >= 100: 
                break;
            else:
                pass

        # Step 3
        # Connect to a database (or create one if it doesn't exist) for saving the list in database
        conn = db.connect('data/sqldb/Ig_Data.db')
        # Create a 'cursor' for executing commands
        c = conn.cursor()

        query_db = ''' SELECT * FROM UsersData'''
        usersDB = pd.read_sql(query_db, conn)

        alreadySaved = usersDB['username'].values

        toCheck = []
        for user in users:
            if user in alreadySaved:
                pass
            else: 
                toCheck.append(user)

        del alreadySaved
        del usersDB
        del data
        del likes

        # Step 4
        ## Login using my account and checking for profile information
        username = key
        _pass = user_pwd[key]

        insta = InstagramBot(username, _pass)
        insta.login()

        for user in toCheck[:100]:
            user, AccountType, posts, following, followers = insta.accountPrivacy(user)
            query_insert = '''INSERT INTO UsersData VALUES ({}{}{},{}{}{},{}{}{},{},{},{})'''\
                        .format("'", today.date() ,"'", "'", user, "'","'", AccountType,"'", posts, following, followers)
            c.execute(query_insert)
            conn.commit()

        # Step 5
        query = '''SELECT * FROM UsersData WHERE date = {}{}{}'''.format("'", today.date(), "'")
        df = pd.read_sql(query, conn)
        # assert len(df) == len(toCheck)
    except TimeoutException:
        pass

C:\Users\PC\Anaconda3\envs\Twitter\lib\site-packages\win_unicode_console\__init__.py:31: RuntimeWarning: sys.stdin.encoding == 'cp1252', whereas sys.stdout.encoding == 'UTF-8', readline hook consumer may assume they are the same
  readline_hook.enable(use_pyreadline=use_pyreadline)
C:\Users\PC\Anaconda3\envs\Twitter\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


2020-04-13 03:24:49
2020-04-13 03:24:45
2020-04-13 03:24:45
2020-04-13 03:24:44
2020-04-13 03:24:42
2020-04-13 03:24:42
2020-04-13 03:24:42
2020-04-13 03:24:41
2020-04-13 03:24:40
2020-04-13 03:24:38
2020-04-13 03:24:38
2020-04-13 03:24:37
2020-04-13 03:24:34
2020-04-13 03:24:29
2020-04-13 03:24:29
2020-04-13 03:24:27
2020-04-13 03:24:26
2020-04-13 03:24:24
2020-04-13 03:24:23
2020-04-13 03:24:23
2020-04-13 03:24:23
2020-04-13 03:24:17
2020-04-13 03:24:15
2020-04-13 03:24:14
2020-04-13 03:24:14
2020-04-13 03:24:09
2020-04-13 03:24:04
2020-04-13 03:24:04
2020-04-13 03:24:03
2020-04-13 03:24:02
2020-04-13 03:24:00
2020-04-13 03:24:00
2020-04-13 03:23:57
2020-04-13 03:23:55
2020-04-13 03:23:55
2020-04-13 03:23:53
2020-04-13 03:23:50
2020-04-13 03:23:49
2020-04-13 03:23:49
2020-04-13 03:23:49
2020-04-13 03:23:47
2020-04-13 03:23:47
2020-04-13 03:23:47
2020-04-13 03:23:43
2020-04-13 03:23:42
2020-04-13 03:23:40
2020-04-13 03:23:39
2020-04-13 03:23:37
2020-04-13 03:23:37
2020-04-13 03:23:35


QueryReturnedNotFoundException: 404 Not Found

In [ ]:
#todo , note all error and put it inside try and except. 

TimeoutException